In [2]:
## import essential modules 
import qumcmc 
from qumcmc.basic_utils import *
from qumcmc.energy_models import IsingEnergyFunction

from qumcmc.classical_mcmc_routines import classical_mcmc
from qumcmc.quantum_mcmc_routines_qulacs import quantum_enhanced_mcmc     #for Qulacs Simulator backend
# from qumcmc.quantum_mcmc_routines_qulacs import quantum_enhanced_mcmc   #for qiskit Aer's Simulator backend 

from qumcmc.trajectory_processing import calculate_running_js_divergence, calculate_running_kl_divergence, calculate_runnning_magnetisation, get_trajectory_statistics

In [3]:
# define the model
np.random.seed(610358)# should always be in the same cell!  
n_spins = 10

## construct problem Hamiltonian ##
shape_of_J=(n_spins,n_spins)

## defining J matrix (mutual 1-1 interaction)
# J =  np.round(np.random.choice([+1, 0, -1], size=(n_spins, n_spins)), decimals=2) 
J =  np.random.uniform(low= -2, high= 2, size= shape_of_J )

J = 0.5 * (J + J.transpose() )
J = np.round( J - np.diag(np.diag(J)) , decimals= 3)

# defining h
h = np.round(0.5 * np.random.randn(n_spins), decimals=2)
#h = np.round(np.random.uniform(low= -1, high = 1, size= (n_spins)), decimals=2)

# instantiate the model
model = IsingEnergyFunction(J, h, name= 'my_model')


In [8]:
@dataclass
class MCMCState:

    def __init__(self, visible:str, hidden:str, accepted:bool= True) -> None:
    
        self.visible = visible
        self.hidden = hidden
        
        accepted: bool
        
        self.bitstring = visible + hidden
    

##### Prepare new data objects

In [17]:
@dataclass
class MCMCState:
    
    visible: str
    hidden: str
    
    accepted: bool = True
    
    def __post_init__(self):
        self.bitstring = self.visible + self.hidden
    

In [18]:
t = MCMCState("100", "11")

In [19]:
print(t)

MCMCState(visible='100', hidden='11', accepted=True)


In [20]:
t.bitstring, t.visible, t.hidden

('10011', '100', '11')